In [2]:
!pip install -U requests beautifulsoup4 -q

In [28]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [4]:
# ~~~ SA BIRD LIST ~~~
####################

def get_sa_bird_list(limit=2):
    # South Africa checklist URL (IOC taxonomy)
    url = "https://avibase.bsc-eoc.org/checklist.jsp?region=za&list=ioc"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    birds = []
    # Birds are usually in table rows within the table
    tables = soup.find_all('table')
    rows = tables[0].find_all('tr', class_="highlight1")
    for row in rows[:limit]:
    # for row in rows[:]:
        cols = row.find_all('td')
        if len(cols) >= 2:
            # Common Name usually in first/second column, Scientific in the other
            common_name = cols[0].get_text(strip=True)
            latin_name = cols[1].find('i').get_text(strip=True) if cols[1].find('i') else cols[1].get_text(strip=True)
            # code
            url = row.find('a')['href']
            code = url[url.find('=')+1:]
            # print(code)
            birds.append({"common": common_name, "latin": latin_name, 'code':code})
    return birds

# bird_list = get_sa_bird_list()
#~~~~~~~~~~~~~~~~~~~~~~

In [48]:
def get_xeno_canto_calls(latin_name):
  # api_url = "https://xeno-canto.org/explore?query=Vanellus%20armatus"
  api_url = "https://xeno-canto.org/explore?query=" + latin_name.replace(' ', '%20')

  response = requests.get(api_url, timeout=10)
  soup = BeautifulSoup(response.text, 'html.parser')
  tables = soup.find_all('table')
  len(tables)

  api_url = "https://xeno-canto.org/explore?query=" + latin_name.replace(' ', '%20')

  response = requests.get(api_url, timeout=10)
  soup = BeautifulSoup(response.text, 'html.parser')
  tables = soup.find_all('table')
  len(tables)

  try:
        # Find the first audio player
    audio_tag = tables[1].find_all('audio', class_="xc-mini-player" )[0]

    # Extract the src attribute
    src = audio_tag['src']

    # Prepend https: if needed
    mp3_url = src if src.startswith("http") else "https:" + src

  except Exception as e:
        # print(f"Error fetching audio for {common_name}, {latin_name}): {e}")     
        mp3_url = 'no audio found'

#   print("MP3 URL:", mp3_url)

  # # Optional: download the file
  # try:
  #     r = requests.get(mp3_url, headers={"User-Agent": "Mozilla/5.0"})
  #     if r.status_code == 200:
  #         with open(latin_name, "wb") as f:
  #             f.write(r.content)
  #         print("Downloaded " + latin_name)
  # except:
  #     with open(latin_name, "wb") as f:
  #         f.write(b"no audio found")

  time.sleep(1)  # Be polite to the server
  return mp3_url

In [26]:
bird_list =get_sa_bird_list(limit=1000)

In [49]:
bird_list =get_sa_bird_list(limit=1000)

audio_df = pd.DataFrame(bird_list, columns=['common', 'latin', 'code', 'audio_url'])

for bird in bird_list:
  common_name = bird['common']
  latin_name = bird['latin']
  print("Fetching audio for:", common_name, latin_name)
  audio_url = get_xeno_canto_calls(latin_name)
  print("Audio URL:", audio_url)
  audio_df.loc[audio_df['latin'] == latin_name, 'audio_url'] = audio_url

audio_df

Fetching audio for: Common Ostrich Struthio camelus
Audio URL: https://xeno-canto.org/sounds/uploaded/WEWZUDAFQF/XC845947-Rattle_(000102_0674).mp3
Fetching audio for: White-faced Whistling Duck Dendrocygna viduata


C:\Users\scuba\AppData\Local\Temp\ipykernel_30400\3068432427.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'https://xeno-canto.org/sounds/uploaded/WEWZUDAFQF/XC845947-Rattle_(000102_0674).mp3' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  audio_df.loc[audio_df['latin'] == latin_name, 'audio_url'] = audio_url


Audio URL: https://xeno-canto.org/sounds/uploaded/THHNKSHUMO/XC1053111-051-Dendrocygne-veuf260127_0474.mp3
Fetching audio for: Fulvous Whistling Duck Dendrocygna bicolor
Audio URL: https://xeno-canto.org/sounds/uploaded/JQSYKZHEMB/XC459633-DendrobicolorTape26LaCandelaria1979.mp3
Fetching audio for: White-backed Duck Thalassornis leuconotus
Audio URL: https://xeno-canto.org/sounds/uploaded/RKQPZQAYOU/XC809424-WBD2.mp3
Fetching audio for: Spur-winged Goose Plectropterus gambensis
Audio URL: https://xeno-canto.org/sounds/uploaded/HZKCNRIPRH/XC126343-SpurWingedGoose21Jan2013BenfontijnFreeStateSouthAfrika.mp3
Fetching audio for: Knob-billed Duck Sarkidiornis melanotos
Audio URL: https://xeno-canto.org/sounds/uploaded/IIJAXHKSCC/XC791481-Knob-billed-duck-%28flight%2C-female%29-KamfinsaDiaryfarm%2CKitwe%2CZAMBIA_20230209_183728.mp3
Fetching audio for: Egyptian Goose Alopochen aegyptiaca
Audio URL: https://xeno-canto.org/sounds/uploaded/YNFBVPUMSY/XC1073916-%E5%9F%83%E5%8F%8A%E9%9B%81%E5%92%8C

,common,latin,code,audio_url
0,Common Ostrich,Struthio camelus,2247CB050A76CF8E,https://xeno-canto.org/sounds/uploaded/WEWZUDA...
1,White-faced Whistling Duck,Dendrocygna viduata,A534AFEA126DBD62,https://xeno-canto.org/sounds/uploaded/THHNKSH...
2,Fulvous Whistling Duck,Dendrocygna bicolor,9A60617EE376ABE9,https://xeno-canto.org/sounds/uploaded/JQSYKZH...
3,White-backed Duck,Thalassornis leuconotus,2EEDACD56C704AE5,https://xeno-canto.org/sounds/uploaded/RKQPZQA...
4,Spur-winged Goose,Plectropterus gambensis,EB01B24C6C4CDD54,https://xeno-canto.org/sounds/uploaded/HZKCNRI...
...,...,...,...,...
891,Ortolan Bunting,Emberiza hortulana,4721056132879EB7,https://xeno-canto.org/sounds/uploaded/DFHRMHO...
892,Lark-like Bunting,Emberiza impetuani,6D102D726F2C2419,https://xeno-canto.org/sounds/uploaded/JDCTKFM...
893,Cinnamon-breasted Bunting,Emberiza tahapisi,0C571D9937B110C8,https://xeno-canto.org/sounds/uploaded/ARLUKQJ...
894,Cape Bunting,Emberiza capensis,DE27F1FA0109531C,https://xeno-canto.org/sounds/uploaded/ODHVIDK...


In [50]:
audio_df.to_csv("xeno-canto_SA_bird_calls.csv", index=False)

# SCRATCH

In [37]:
latin_name = bird['latin']
common_name = bird['common']
print("Fetching audio for:", common_name, latin_name)    

Fetching audio for: Southern Crested Guineafowl Guttera edouardi


In [44]:
api_url = "https://xeno-canto.org/explore?query=" + latin_name.replace(' ', '%20')

response = requests.get(api_url, timeout=10)
soup = BeautifulSoup(response.text, 'html.parser')
tables = soup.find_all('table')
len(tables)

try:
      # Find the first audio player
  audio_tag = tables[1].find_all('audio', class_="xc-mini-player" )[0]

  # Extract the src attribute
  src = audio_tag['src']

  # Prepend https: if needed
  mp3_url = src if src.startswith("http") else "https:" + src

except Exception as e:
      # print(f"Error fetching audio for {common_name}, {latin_name}): {e}")     
      mp3_url = 'no audio found'

mp3_url

'no audio found'

In [47]:
# Optional: download the file
try:
    r = requests.get(mp3_url, headers={"User-Agent": "Mozilla/5.0"})
    if r.status_code == 200:
        with open(latin_name, "wb") as f:
            f.write(r.content)
        print("Downloaded " + latin_name)
except:
    with open(latin_name, "wb") as f:
        f.write(b"no audio found")

In [19]:
bird_list =get_sa_bird_list()
latin_name =bird_list[1]['latin']
latin_name

'Dendrocygna viduata'

In [24]:
audio_url = get_xeno_canto_calls(latin_name)
audio_url

'https://xeno-canto.org/sounds/uploaded/THHNKSHUMO/XC1053111-051-Dendrocygne-veuf260127_0474.mp3'

In [ ]:
for bird in bird_list:
  latin_name = bird['latin']
  print("Fetching audio for:", latin_name)
  audio_url = get_xeno_canto_calls(latin_name)
  print("Audio URL:", audio_url)

Fetching audio for: Struthio camelus
Audio URL: https://xeno-canto.org/sounds/uploaded/WEWZUDAFQF/XC845947-Rattle_(000102_0674).mp3
Fetching audio for: Dendrocygna viduata
Audio URL: https://xeno-canto.org/sounds/uploaded/THHNKSHUMO/XC1053111-051-Dendrocygne-veuf260127_0474.mp3


In [30]:
bird_list =get_sa_bird_list(limit=1000)
audio_df = pd.DataFrame(bird_list, columns=['common', 'latin', 'code', 'audio_url'])
audio_df

,common,latin,code,audio_url
0,Common Ostrich,Struthio camelus,2247CB050A76CF8E,NaN
1,White-faced Whistling Duck,Dendrocygna viduata,A534AFEA126DBD62,NaN
2,Fulvous Whistling Duck,Dendrocygna bicolor,9A60617EE376ABE9,NaN
3,White-backed Duck,Thalassornis leuconotus,2EEDACD56C704AE5,NaN
4,Spur-winged Goose,Plectropterus gambensis,EB01B24C6C4CDD54,NaN
...,...,...,...,...
891,Ortolan Bunting,Emberiza hortulana,4721056132879EB7,NaN
892,Lark-like Bunting,Emberiza impetuani,6D102D726F2C2419,NaN
893,Cinnamon-breasted Bunting,Emberiza tahapisi,0C571D9937B110C8,NaN
894,Cape Bunting,Emberiza capensis,DE27F1FA0109531C,NaN
